# DAY1

In [30]:
import torch
import torchvision
import torchvision.datasets as Datasets
import torch.nn as nn
import torch.autograd as AG

In [7]:
torch.manual_seed(777)
train_data = Datasets.MNIST(root='MNIST_data/',
                           train = True,
                           transform=torchvision.transforms.ToTensor(),
                           download=True)
test_data = Datasets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=torchvision.transforms.ToTensor(),
                          download=True)

In [15]:
def data_loader(data_set,batch_size):
    dataloader = torch.utils.data.DataLoader(dataset=data_set,batch_size=batch_size,shuffle=True)
    return data_loader

In [16]:
train_loader = data_loader(train_data,5)

In [29]:
# for getting image details
train_image,train_target = train_data[0]
train_image.shape

torch.Size([1, 28, 28])

In [33]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        #Input channels = 1, output channels = 32 
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = 1,out_channels = 32,kernel_size = 3 ,stride = 1 ,padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        #Fully Connected Layer Next with 7x7x64 inputs to 10 outputs
        self.FC = nn.Linear(in_features=64*7*7,out_features=10,bias=True)
        nn.init.xavier_uniform_(self.FC.weight)
        
    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(layer2)
        layer2 = layer2.view(layer2.size(0),-1)
        out = self.FC(layer2)
        return out
        

In [34]:
# hyperparameters 
learning_rate = 0.001
training_epochs = 5
batch_size = 100
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),learning_rate)

# DAY2